In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math

from torch import optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import argparse
import warnings
import os, sys
import time
from scipy import stats





In [ ]:
import torch.nn.functional as F
# model input is strictly enforced as (B x C x T). Output size is strictly enforced as (B x T)
class MLP(nn.Module):
  def __init__(self, input_size, channel_size,hidden_size, output_size):
    super(MLP, self).__init__()
    self.flat = nn.Flatten()
    self.fc1 = nn.Linear(input_size*channel_size, hidden_size) # First layer
    self.relu = nn.ReLU() # relu activation function
    self.fc2 = nn.Linear(hidden_size, output_size) # Second layer

  # Forward is equivalent to the first equation in the paper
  def forward(self, x):
    x = self.flat(x)
    x = self.fc1(x) # we take the output of layer 1 (fc1)
    x = self.relu(x) # run the output through the relu activation
    x = self.fc2(x) # then run that throught the last layer
    return x

input_size = 48
hidden_size = 128
channel_size = 11
output_size = 6

model = MLP(input_size, channel_size, hidden_size, output_size).cuda()

#Main Function

In [ ]:
data = np.loadtxt('demo_data.csv',delimiter=',')

adding a zero vector attached to data to create a 52608 x 7 vector

In [ ]:
# Split the time series into training, validation and testing series
train_portion = 0.48
vali_portion = 0.4
test_portion = 1- train_portion - vali_portion

train_size = np.ceil(train_portion*data.shape[0]).astype('int')
vali_size = np.ceil(vali_portion*data.shape[0]).astype('int')

data_train = data[:train_size,:]
data_validation = data[train_size:train_size+vali_size,:]
data_test = data[train_size+vali_size:,:]

def normalize_data(data):
    return data,0,0

def generate_forecast_data(train_data,train_window_size=750,predict_window_size = 200):
  train_data = torch.from_numpy(train_data)
  train_data,mean_data, std_data = normalize_data(train_data)
  data_segments = train_data.unfold(dimension = 0,size = train_window_size, step = 1).float()
  target_data = train_data[train_window_size:]
  target_data_segments = target_data.unfold(dimension = 0,size = predict_window_size, step = 1).float()
  data_segments = data_segments[0:target_data_segments.size(0),:]
  target_data_segments = target_data_segments[:,[0,-1],:]
  return data_segments, target_data_segments, mean_data, std_data

# split data and save mean, sd.
predict_window_size = 6
train_window_size = 48


train,target,mean_data, std_data = generate_forecast_data(data_train,train_window_size=train_window_size, predict_window_size=predict_window_size)
test,target_test,mean_data2, std_data2 = generate_forecast_data(data_test,train_window_size=train_window_size, predict_window_size=predict_window_size)

# add validation and validation target ect.
validation,target_validation,mean_data3,std_data3 = generate_forecast_data(data_validation,train_window_size=train_window_size, predict_window_size=predict_window_size)


# Convert data to PyTorch tensors
train_tensor = torch.tensor(train, dtype=torch.float32)
test_tensor = torch.tensor(test, dtype=torch.float32)
target_tensor = torch.tensor(target, dtype=torch.float32)
target_test_tensor = torch.tensor(target_test, dtype=torch.float32)

# make the new validations into tensors
validation_tensor = torch.tensor(validation, dtype=torch.float32)
target_validation_tensor = torch.tensor(target_validation, dtype=torch.float32)






<ipython-input-22-3f67573e9285>:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_tensor = torch.tensor(train, dtype=torch.float32)
<ipython-input-22-3f67573e9285>:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_tensor = torch.tensor(test, dtype=torch.float32)
<ipython-input-22-3f67573e9285>:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target_tensor = torch.tensor(target, dtype=torch.float32)
<ipython-input-22-3f67573e9285>:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().de

In [ ]:
train_y = target_tensor.cuda()
train_X = train_tensor.cuda()
valid_X = validation_tensor.cuda()
valid_y = target_validation_tensor.cuda()
test_X = test_tensor.cuda()
test_Y = target_test_tensor.cuda()

In [ ]:
train_X = train_X[:,1:,:]
valid_X = valid_X[:,1:,:]
test_X = test_X[:,1:,:]
train_y = train_y[:,1,:]
valid_y = valid_y[:,1,:]
test_y = test_Y[:,1,:]

In [ ]:
train_X.shape

torch.Size([12573, 11, 48])

In [ ]:
criterion = nn.MSELoss()

model_optim = optim.Adam(model.parameters(), lr=1e-4)


In [ ]:
model = model.cuda()

In [ ]:
loss_train = 0;
# Forward pass

for i in range(2000):
    model.train()
    model_optim.zero_grad()
    outputs = model(train_X)
    loss = criterion(outputs, train_y)
    loss.backward()
    model_optim.step()
    print(loss)


tensor(0.6242, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.5954, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.5686, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.5440, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.5213, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.5006, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.4816, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.4643, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.4484, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.4339, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.4207, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.4085, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.3974, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.3872, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.3777, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.3691, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.3611, device='cuda:0', grad_fn=